In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [2]:
# get instance

s = server.Server(
    f"node-persist-project46", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server node-persist-project46's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-project46
Id,844de1a7-42eb-41aa-bf2e-95d20b28168d
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.2.253 (v4) Type: fixed MAC: fa:16:3e:fe:82:7c IP: 129.114.25.29 (v4) Type: floating MAC: fa:16:3e:fe:82:7c
Network Name,sharednet1
Created At,2025-05-09T20:02:25Z
Keypair,tc4038_nyu_edu-jupyter
Reservation Id,None
Host Id,f9cc22c1cd59daa26bc3d78a0c7365032951b724a1e82c861b9b1802


In [3]:
s.associate_floating_ip()

ResourceError: None of the ports can route to floating ip 129.114.27.7 on server 844de1a7-42eb-41aa-bf2e-95d20b28168d

In [4]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.25.29 port 22.


Connection successful


In [5]:
# security group

security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"},
  {'name': "allow-8080", 'port': 8080, 'description': "Enable TCP port 8080 (used by AirFlow)"}
]

os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:
  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8080', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [6]:
# set up docker

s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.25.29: b'c611879826a97aa26c0e0daaeb5b976a'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

## Create object storage

### Set up rclone
Open the GUI for CHI@TACC:
- from the Chameleon website
- click “Experiment” > “CHI@TACC”
- log in if prompted to do so
- check the project drop-down menu near the top left (which shows e.g. “CHI-XXXXXX”), and make sure the correct project is selected.

In the menu sidebar on the left side, click on “Object Store” > “Containers” and then, “Create Container”. 
- Specify the name 
- Leave other settings at their defaults, and click “Submit”.

In the menu sidebar on the left side of the Horizon GUI, click “Identity” > “Application Credentials”. Then, click “Create Application Credential”.
- In the “Name”, field, use “data-persist”.
- Set the “Expiration” date to the end date of the current semester. (Note that this will be in UTC time, not your local time zone.) This ensures that if your credential is leaked (e.g. you accidentially push it to a public Github repository), the damage is mitigated.
- Click “Create Application Credential”.
- Copy the “ID” and “Secret” displayed in the dialog, and save them in a safe place. You will not be able to view the secret again from the Horizon GUI. Then, click “Download openrc file” to have another copy of the secret.

Back to your instance,

``` bash
# run on node-persist
curl https://rclone.org/install.sh | sudo bash
``` 

``` bash
# run on node-persist
# this line makes sure user_allow_other is un-commented in /etc/fuse.conf
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf
``` 

``` bash
# run on node-persist
mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf
``` 

Fill in your application_credential
```bash
[chi_tacc]
type = swift
user_id = YOUR_USER_ID
application_credential_id = APP_CRED_ID
application_credential_secret = APP_CRED_SECRET
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
```


### Create ETL pipeline to load data into the object store

Get data online and laod into our object storage. 

``` bash
# run on node-persist
docker compose -f docker-compose-etl.yaml run extract-data
``` 

``` bash
# run on node-persist
docker compose -f docker-compose-etl.yaml run transform-data
``` 

Replace with your container name
``` bash
# run on node-persist
export RCLONE_CONTAINER=object-persist-project46
docker compose -f docker-compose-etl.yaml run load-data
``` 

``` bash
# run on node-persist
docker volume rm food11-etl_food11
``` 


### Mount the object store

``` bash
# run on node-persist
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object
``` 

Replace with your container name
``` bash
# run on node-persist
rclone mount chi_tacc:object-persist-project46 /mnt/object --read-only --allow-other --daemon
``` 

Run
``` bash
# run on node-persist
ls /mnt/object
``` 
to confirm there is the data directories. 

In [3]:
# create volume

cinder_client = chi.clients.cinder()
cinder_client.volumes.list()
volume = cinder_client.volumes.create(name=f"block-persist-project46", size=2)
volume._info

{'id': '9a872ab0-dd2a-4bcc-8e14-1067e533cc7f',
 'status': 'creating',
 'size': 2,
 'availability_zone': 'nova',
 'created_at': '2025-05-10T19:29:54.476752',
 'updated_at': None,
 'name': 'block-persist-project46',
 'description': None,
 'volume_type': 'ceph-hdd',
 'snapshot_id': None,
 'source_volid': None,
 'metadata': {},
 'links': [{'rel': 'self',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/v3/89f528973fea4b3a981f9b2344e522de/volumes/9a872ab0-dd2a-4bcc-8e14-1067e533cc7f'},
  {'rel': 'bookmark',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/89f528973fea4b3a981f9b2344e522de/volumes/9a872ab0-dd2a-4bcc-8e14-1067e533cc7f'}],
 'user_id': '8125dddc60b12b71d1538f9444d174e37e647052370f871a725e4861e482938a',
 'bootable': 'false',
 'encrypted': False,
 'replication_status': None,
 'consistencygroup_id': None,
 'multiattach': False,
 'attachments': []}

In [4]:
# attach to instance

volume_name = 'block-persist-project46'

cinder_client = chi.clients.cinder()
volume = [v for v in cinder_client.volumes.list() if v.name== volume_name][0]

volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id = s.id, volume_id = volume.id)

<Volume: 9a872ab0-dd2a-4bcc-8e14-1067e533cc7f>

On the instance, let’s confirm that we can see the block storage volume. Run

``` bash
# run on node-persist
lsblk
```

and verify that `vdb` appears in the output.


``` bash
# run on node-persist
sudo parted -s /dev/vdb mklabel gpt
sudo parted -s /dev/vdb mkpart primary ext4 0% 100%
```

Verify that we now have the partition `vdb1` in the output of

``` bash
# run on node-persist
lsblk
```

Next, we format the partition:

``` bash
# run on node-persist
sudo mkfs.ext4 /dev/vdb1
```

Finally, create a directory in the local filesystem, mount the partition to that directory:

``` bash
# run on node-persist
sudo mkdir -p /mnt/block
sudo mount /dev/vdb1 /mnt/block
```

and change the owner of that directory to the `cc` user:

``` bash
# run on node-persist
sudo chown -R cc /mnt/block
sudo chgrp -R cc /mnt/block
```

Run

``` bash
# run on node-persist
df -h
```

and verify that the output includes a line with `/dev/vdb1` mounted on `/mnt/block`:

    Filesystem      Size  Used Avail Use% Mounted on
    /dev/vdb1       2.0G   24K  1.9G   1% /m